<a href="https://colab.research.google.com/github/AgneloLi/Projeto_Triagem_Infarto_PS/blob/main/Projeto_Triagem_Infarto_PS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
!pip install -q -U google-generativeai

In [72]:
import google.generativeai as genai
import pandas as pd
import numpy as np
from numpy.linalg import norm

from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)


In [73]:
# Configuração do modelo
generation_config = {
    "candidate_count": 1,
    "temperature": 0.5
}
safety_settings = {
    "HARASSMENT": "BLOCK_NONE",
    "HATE": "BLOCK_NONE",
    "SEXUAL": "BLOCK_NONE",
    "DANGEROUS": "BLOCK_NONE"
}

In [74]:
model = genai.GenerativeModel(model_name="gemini-1.0-pro", generation_config=generation_config, safety_settings=safety_settings)
modelEmb = "models/embedding-001"  # Modelo de embedding

In [135]:
# Base de dados de sintomas
sintomas_data = [
    {
        "Título": """--------CERTEZA DE INFARTO DO MIOCÁRIDO--------
        """,
        "Conteúdo": """*************URGENTE************
        Leve o paciente até a sala de EmergÊncia. Monitore os Sinais vitais, oxigênio e eletrocardiograma até o médico chegar.""",
        "Palavras-chave": ["10", "9", "8",  "forte", "intensa", "dor no braço", "mandíbula", "falta de ar", "pescoço", "náuseas sim", "suor frio", "adormecido", "dormente", "intensa", "irradia", "sudorese", "tontura", "palpitação", "palidez", "persistente"]
    },
    {
        "Título": """--------SUEPEITA DE INFARTO DO MIOCÁRIDO--------
        """,
        "Conteúdo": """------****SUSPEITA****-------
        Leve imediatamente o paciente até a sala de emergência, ofereça apoio emocional até a chegada do médico. Evite falar sobre assuntos que causem estresse ou ansiedade. Preste atenção à pressão arterial, frequência cardíaca e respiração.""",
        "Palavras-chave": ["7", "6", "5", "fraca","náuseas sim", "pouca", "desconforto", "queimação", "indigestão", "fadiga", "irradiação", "falta de ar"]
    },
    {
        "Título": """--------SEM SUSPEITA DE INFARTO DO MIOCÁRIDO--------
        """,
        "Conteúdo": """**Sem Perigo Iminente**
        O paciente pode aguardar atendimento com os demais. Fique atento em caso de mudanças na situação clínica. Em caso de alteração, refaça o chat""",
        "Palavras-chave": ["4", "3", "2", "1", "0", "sem","sem náuseas", "minima", "gases", "dor leve", "intermitente"]
    }
]

In [136]:

df = pd.DataFrame(sintomas_data)

def calcular_similaridade(respostas, sintomas_data):
    # Tokenizar respostas
    tokens = []
    for resposta in respostas:
        tokens.extend(resposta.split())

    # Criar conjunto de todas as palavras-chave
    palavras_chave = set()
    for documento in sintomas_data:
        palavras_chave.update(documento['Palavras-chave'])

    # Calcular similaridade
    similaridade = 0
    for token in tokens:
        if token in palavras_chave:
            similaridade += 1

    return similaridade

def encontrar_documento_similar(respostas, sintomas_data):
    max_similaridade = 0
    indice_mais_similar = -1

    for i, documento in df.iterrows():
        similaridade = calcular_similaridade(respostas, [documento])  # Calcula a similaridade para cada documento

        if similaridade > max_similaridade:
            max_similaridade = similaridade
            indice_mais_similar = i

    return indice_mais_similar

In [137]:
print("Chatbot de Triagem Hospitalar:")
while True:
    # forte, fraca ou sem
    intensidade = input("1. Sente dor no peito ou nas costas? ")
    # De 0 à 10
    dor_peito = input("2. Qual a intensidade da dor no peito? ")
    # intensa,pouca ou minima
    falta_ar = input("3. Está com falta de ar? ").lower()
    nauseas = input("4. Sente náuseas? (sim/não) ").lower()
    suor_frio = input("5. Tem suor frio? (sim/não) ").lower()
    dor_braco = input("6. Sente dor no braço esquerdo? (sim/não) ").lower()

    respostas = [f"{intensidade}", f"{dor_peito}", falta_ar, f"náuseas {nauseas}", f"suor frio {suor_frio}", f"dor no braço {dor_braco}"]

    indice_documento_similar = encontrar_documento_similar(respostas, sintomas_data)
    documento_similar = df.iloc[indice_documento_similar]

    print("\nDocumento mais similar:")
    print(f"Título: {documento_similar['Título']}")
    print(f"Conteúdo: {documento_similar['Conteúdo']}\n")

    continuar = input("Deseja triar outro paciente? (s/n) ")
    if continuar.lower() != 's':
        break

Chatbot de Triagem Hospitalar:
1. Sente dor no peito? forte
2. Qual a intensidade da dor no peito? 10
3. Está com falta de ar? sim
4. Sente náuseas? (sim/não) sim
5. Tem suor frio? (sim/não) sim
6. Sente dor no braço esquerdo? (sim/não) sim

Documento mais similar:
Título: --------CERTEZA DE INFARTO DO MIOCÁRIDO--------
        
Conteúdo: *************URGENTE************
        Leve o paciente até a sala de EmergÊncia. Monitore os Sinais vitais, oxigênio e eletrocardiograma até o médico chegar.

Deseja triar outro paciente? (s/n) s
1. Sente dor no peito? fraca
2. Qual a intensidade da dor no peito? 5
3. Está com falta de ar? nao
4. Sente náuseas? (sim/não) sim
5. Tem suor frio? (sim/não) nao
6. Sente dor no braço esquerdo? (sim/não) nao

Documento mais similar:
Título: --------SUEPEITA DE INFARTO DO MIOCÁRIDO--------
        
Conteúdo: ------****SUSPEITA****-------
        Leve imediatamente o paciente até a sala de emergência, ofereça apoio emocional até a chegada do médico. Evite fa